## Tổng quan

- Sau khi đã khám phá dữ liệu ban đầu thì chúng ta nhận thấy dataset còn nhiều missing value cần phải giải quyết cũng như là còn một số cột chứa giá trị chưa thống nhất nhau (`experience`, `company_size`, `last_new_job`).
- Bước tiếp theo sẽ là tiền xử lý dữ liệu để đảm bảo dữ liệu sạch, có thể đem đi train mô hình và trả lời các câu hỏi đặt ra chuẩn xác hơn.

## Setup đường dẫn

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  

## Import các thư viện và module

In [2]:
import numpy as np

import src.data_processing as dp

## Load dữ liệu

Dữ liệu sẽ được đọc từ file `aug_train.csv`. Các features được lưu vào header, toàn bộ dữ liệu sẽ được lưu vào data.

In [3]:
input_path = "../data/raw/aug_train.csv"

header = dp.read_header(input_path) 
data = dp.read_data(input_path) 
col_idx = {name: i for i, name in enumerate(header)} # Mỗi header tương ứng với 1 index để dễ truy cập  


print("Features:")
print(header)
print("\nData tương ứng:")
print(data[:5,:])

Features:
['enrollee_id' 'city' 'city_development_index' 'gender'
 'relevent_experience' 'enrolled_university' 'education_level'
 'major_discipline' 'experience' 'company_size' 'company_type'
 'last_new_job' 'training_hours' 'target']

Data tương ứng:
[['8949' 'city_103' '0.92' 'Male' 'Has relevent experience'
  'no_enrollment' 'Graduate' 'STEM' '>20' 'nan' 'nan' '1' '36' '1.0']
 ['29725' 'city_40' '0.7759999999999999' 'Male' 'No relevent experience'
  'no_enrollment' 'Graduate' 'STEM' '15' '50-99' 'Pvt Ltd' '>4' '47'
  '0.0']
 ['11561' 'city_21' '0.624' 'nan' 'No relevent experience'
  'Full time course' 'Graduate' 'STEM' '5' 'nan' 'nan' 'never' '83' '0.0']
 ['33241' 'city_115' '0.789' 'nan' 'No relevent experience' 'nan'
  'Graduate' 'Business Degree' '<1' 'nan' 'Pvt Ltd' 'never' '52' '1.0']
 ['666' 'city_162' '0.767' 'Male' 'Has relevent experience'
  'no_enrollment' 'Masters' 'STEM' '>20' '50-99' 'Funded Startup' '4' '8'
  '0.0']]


## Tiền xử lý dữ liệu

### Xóa các cột không cần thiết

In [ ]:
# Xóa 2 cột 'enrollee_id' và 'gender'
index_to_delete = [col_idx['enrollee_id'], col_idx['gender']]
data = np.delete(data, index_to_delete, axis=1)

# Cập nhật lại header và col_idx
header = new_arr = np.delete(header, index_to_delete)
col_idx = {name: i for i, name in enumerate(header)} 

{np.str_('city'): 0,
 np.str_('city_development_index'): 1,
 np.str_('relevent_experience'): 2,
 np.str_('enrolled_university'): 3,
 np.str_('education_level'): 4,
 np.str_('major_discipline'): 5,
 np.str_('experience'): 6,
 np.str_('company_size'): 7,
 np.str_('company_type'): 8,
 np.str_('last_new_job'): 9,
 np.str_('training_hours'): 10,
 np.str_('target'): 11}

### Xử lý với mỗi cột có kiểu dữ liệu dạng số (numerical)   

Để giải quyết các giá trị bị thiếu của mỗi cột có kiểu dữ liệu dạng số, chúng ta có thể thực hiện như sau:

- Đối với cột `experience`: chúng ta thấy giá trị của cột này còn chứa các giá trị chuỗi như '<1' và '>20' — trước tiên, hãy chuyển đổi tất cả các giá trị này sang định dạng số phù hợp. Các giá trị thiếu sẽ được thay thế bằng giá trị trung vị.
- Đối với cột `company_size`: chúng ta thấy giá trị của cột này còn chứa các giá trị chuỗi như '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10/49', '100-500', '500-999' — trước tiên, hãy chuyển đổi tất cả các giá trị này sang định dạng số phù hợp. Các giá trị thiếu sẽ được thay thế bằng giá trị trung vị.
- Đối với cột `last_new_job`: chúng ta thấy giá trị của cột này còn chứa các giá trị chuỗi như '>4' và 'never' — trước tiên, hãy chuyển đổi tất cả các giá trị này sang định dạng số phù hợp. Các giá trị thiếu sẽ được thay thế bằng giá trị trung vị.

In [5]:
# Chuyển đổi các giá trị không hợp lý
numeric_transformers = [
    ('experience', dp.convert_experience),
    ('company_size', dp.convert_company_size),
    ('last_new_job', dp.convert_last_new_job)
]
for col_name, convert in numeric_transformers:
    idx = col_idx[col_name]
    
    col_float = convert(data[:, idx])
    
    median_val = np.nanmedian(col_float)
    col_float[np.isnan(col_float)] = median_val
    
    # data đang là mảng string, nên phải ép kiểu số về string trước khi lưu lại vào data
    data[:, idx] = col_float.astype(str)

### Xử lý với mỗi cột có kiểu dữ liệu dạng phân loại (categorical)

Để giải quyết các giá trị bị thiếu của mỗi cột có kiểu dữ liệu dạng phân loại, chúng ta có thể thực hiện như sau:

- Đối với các cột `enrolled_university`, `education_level`, `major_discipline`, `company_type`: các giá trị thiếu được thay thế bằng 'Unknown' 

In [6]:
categorical_cols = ['enrolled_university', 'education_level', 
                    'major_discipline', 'company_type']

cat_idxs = [col_idx[name] for name in categorical_cols]

sub_data = data[:, cat_idxs]

sub_data[sub_data == 'nan'] = 'Unknown'

# Gán vào lại data
data[:, cat_idxs] = sub_data


### Xử lý giá trị ngoại lai cho cột `training_hours`

In [7]:
idx_hours = col_idx['training_hours']
data[:, idx_hours] = dp.handle_outliers_iqr(data[:, idx_hours])

### Dữ liệu sau khi xử lý missing values sẽ như thế nào?

In [8]:
for h in header:
    print(f"Cột {h}:")
    print(np.unique(data[:, col_idx[h]]))

Cột city:
['city_1' 'city_10' 'city_100' 'city_101' 'city_102' 'city_103' 'city_104'
 'city_105' 'city_106' 'city_107' 'city_109' 'city_11' 'city_111'
 'city_114' 'city_115' 'city_116' 'city_117' 'city_118' 'city_12'
 'city_120' 'city_121' 'city_123' 'city_126' 'city_127' 'city_128'
 'city_129' 'city_13' 'city_131' 'city_133' 'city_134' 'city_136'
 'city_138' 'city_139' 'city_14' 'city_140' 'city_141' 'city_142'
 'city_143' 'city_144' 'city_145' 'city_146' 'city_149' 'city_150'
 'city_152' 'city_155' 'city_157' 'city_158' 'city_159' 'city_16'
 'city_160' 'city_162' 'city_165' 'city_166' 'city_167' 'city_171'
 'city_173' 'city_175' 'city_176' 'city_179' 'city_18' 'city_180'
 'city_19' 'city_2' 'city_20' 'city_21' 'city_23' 'city_24' 'city_25'
 'city_26' 'city_27' 'city_28' 'city_30' 'city_31' 'city_33' 'city_36'
 'city_37' 'city_39' 'city_40' 'city_41' 'city_42' 'city_43' 'city_44'
 'city_45' 'city_46' 'city_48' 'city_50' 'city_53' 'city_54' 'city_55'
 'city_57' 'city_59' 'city_61' 'cit

## Feature engineering 

Tạo thêm các feature mới:
- `is_active_learner`: Đây là tín hiệu hành vi rõ ràng nhất. Một người đang đi học (Full time/Part time) thì khả năng cực cao họ sẽ tìm việc mới sau khi tốt nghiệp. Ngược lại, người no_enrollment thường đang ổn định.
- `experience_level`:  Hành vi của Junior và Senior rất khác nhau. Junior hay nhảy việc để tăng lương/học hỏi. Senior thường ngại thay đổi. Việc chia nhóm giúp Model bắt được quy luật này tốt hơn là để số lẻ.
- `is_stem_major`: Vì là tuyển dụng Data Scientist. Người có nền tảng STEM (Toán, Kỹ thuật) là đối tượng chính. Những ngành khác (Arts, Humanities) thường là chuyển ngành, hành vi tìm việc của họ sẽ khác.

In [9]:
# Feature: is_stem_major
raw_major = data[:, col_idx['major_discipline']]
is_stem = dp.create_is_stem(raw_major) 

# Feature: experience_level (0: Junior, 1: Mid, 2: Senior)
exp_float = data[:, col_idx['experience']].astype(float)
exp_level = dp.create_experience_level(exp_float)

# Feature: is_active_learner (Đang đi học)
raw_enroll = data[:, col_idx['enrolled_university']]
is_learner = dp.create_enrollment_status(raw_enroll)

new_features = np.column_stack((is_stem, exp_level, is_learner))

data = np.hstack((data, new_features.astype(str)))

header = np.append(header, ['is_stem_major', 'experience_level', 'is_active_learner'])


## Save dữ liệu đã xử lý

In [10]:
output_path = "../data/processed/aug_train_processed.csv"

dp.save_to_csv(data, header, output_path)

✅ Đã lưu file thành công tại: ../data/processed/aug_train_processed.csv
